In [1]:
import os
os.getcwd()

'/Users/Young/projects/house_crawling'

In [2]:
!pip install bs4
!pip install webdriver_manager
!pip install selenium
!pip install random_user_agent



In [3]:
from selenium import webdriver
import selenium
import random_user_agent

from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType 

from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

from time import sleep
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import json
import re




In [55]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

chrome_options = Options()
chrome_options.add_argument('--incognito')
# chrome_options.add_argument('headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument(f'user-agent={user_agent}')
# chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])
# chrome_options.add_argument("--disable-blink-features");
# chrome_options.add_argument("--disable-blink-features=AutomationControlled");
# driver = webdriver.Chrome(options=chrome_options)
executable_path=os.getcwd()+'/chromedriver'
#         print(executable_path)
# browser = webdriver.Chrome(executable_path=executable_path, options=chrome_options)
# browser = webdriver.Chrome(executable_path="./chromedriver", options=chrome_options)


# browser = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)

from webdriver_manager.utils import ChromeType
browser = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install(), options=chrome_options)


# from webdriver_manager.chrome import ChromeDriverManager
# from webdriver_manager.utils import ChromeType


# browser = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install())


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [/Users/Young/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [56]:
# url = "https://adl.edu.tw/modules.php?op=modload&name=assignMission&file=ks_viewlist&seme=109&subject=28&subjectG=2#parentHorizontalTab1"

url = "https://adl.edu.tw/"

# _ga=GA1.3.1985953389.1619174961; 
# _gid=GA1.3.1303494072.1619174961; 
# G_ENABLED_IDPS=google; 
# set_school0=%E8%8B%97%E6%A0%97%E7%B8%A3; 
# set_school1=358-%E8%8B%91%E8%A3%A1%E9%8E%AE; 
# set_school2=054510-%E8%87%B4%E6%B0%91%E5%9C%8B%E4%B8%AD; 
# PHPSESSID=089a4f4a7e7fc248ac07012d471d3051; 
# id_level=1; 
# school0=%E8%8B%97%E6%A0%97%E7%B8%A3; 
# school1=358-%E8%8B%91%E8%A3%A1%E9%8E%AE; 
# school2=054510-%E8%87%B4%E6%B0%91%E5%9C%8B%E4%B8%AD; 
# username=054510-1638601; 
# captcha_code=4565; 
# authchallenge=20e91cea651b564b713dce622c7edf52; 
# user_enter=NoshowDescription; 
# _gat_gtag_UA_108376301_1=1

browser.get(url)
# browser.add_cookie({"name": "_ga", "value": "GA1.3.1985953389.1619174961"})
# browser.add_cookie({"name": "_gid", "value": "GA1.3.1303494072.1619174961"})
browser.add_cookie({"name": "G_ENABLED_IDPS", "value": "google"})
# browser.add_cookie({"name": "set_school0", "value": "%E8%8B%97%E6%A0%97%E7%B8%A3"})
# browser.add_cookie({"name": "set_school1", "value": "358-%E8%8B%91%E8%A3%A1%E9%8E%AE"})
# browser.add_cookie({"name": "set_school2", "value": "054510-%E8%87%B4%E6%B0%91%E5%9C%8B%E4%B8%AD"})
browser.add_cookie({"name": "PHPSESSID", "value": "089a4f4a7e7fc248ac07012d471d3051"})
# browser.add_cookie({"name": "id_level", "value": "1"})
# browser.add_cookie({"name": "school0", "value": "%E8%8B%97%E6%A0%97%E7%B8%A3"})
# browser.add_cookie({"name": "school1", "value": "358-%E8%8B%91%E8%A3%A1%E9%8E%AE"})
# browser.add_cookie({"name": "school2", "value": "054510-%E8%87%B4%E6%B0%91%E5%9C%8B%E4%B8%AD"})
# browser.add_cookie({"name": "username", "value": "054510-1638601"})
browser.add_cookie({"name": "authchallenge", "value": "20e91cea651b564b713dce622c7edf52"})
# browser.add_cookie({"name": "user_enter", "value": "NoshowDescription"})
# browser.add_cookie({"name": "_gat_gtag_UA_108376301_1", "value": "1"})


In [54]:
print(browser.get_cookies())
# print(browser.get_cookie("foo"))


[{'domain': 'adl.edu.tw', 'httpOnly': False, 'name': '_gat_gtag_UA_108376301_1', 'path': '/', 'secure': True, 'value': '1'}, {'domain': 'adl.edu.tw', 'httpOnly': False, 'name': 'user_enter', 'path': '/', 'secure': True, 'value': 'NoshowDescription'}, {'domain': 'adl.edu.tw', 'httpOnly': False, 'name': 'authchallenge', 'path': '/', 'secure': True, 'value': '20e91cea651b564b713dce622c7edf52'}, {'domain': 'adl.edu.tw', 'httpOnly': False, 'name': 'school2', 'path': '/', 'secure': True, 'value': '054510-%E8%87%B4%E6%B0%91%E5%9C%8B%E4%B8%AD'}, {'domain': 'adl.edu.tw', 'httpOnly': False, 'name': 'username', 'path': '/', 'secure': True, 'value': '054510-1638601'}, {'domain': 'adl.edu.tw', 'httpOnly': False, 'name': 'school1', 'path': '/', 'secure': True, 'value': '358-%E8%8B%91%E8%A3%A1%E9%8E%AE'}, {'domain': '.adl.edu.tw', 'expiry': 1619427285, 'httpOnly': False, 'name': '_gid', 'path': '/', 'secure': False, 'value': 'GA1.3.652978620.1619340886'}, {'domain': '.adl.edu.tw', 'expiry': 161934094

In [5]:
# browser.add_cookie({"name": "foo", "value": "bar"})

url = "https://rent.591.com.tw/?kind=0&region=1&rentprice=5&shape=2&other=lift,cartplace,cook"


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=90.0.4430.85)


In [10]:
print(f"===fetch:{url}===")

pattern = r".*&region=(\d).*"
region = re.match(pattern, url).group(1)

print(f"region={region}")

# engine = selenium_engine()
# browser = engine.browser
browser.get(url)
timeout = 3
try:
    element_present = EC.presence_of_element_located((By.ID, "area-box-close"))
    WebDriverWait(browser, timeout).until(element_present)
    browser.find_element_by_css_selector(f'[data-id="{region}"]').click()
except TimeoutException:
    print("Timed out waiting for page to load")
finally:
    print("Page loaded")

timeout = 10
try:
    element_present = EC.presence_of_element_located((By.CLASS_NAME, "pageBar"))
    WebDriverWait(browser, timeout).until(element_present)
except TimeoutException:
    print("Timed out waiting for page to load")
finally:
    print("Page loaded")    

sleep(5)    

data_soup_list = []
last_page = False
while last_page is False:
    data_soup = BeautifulSoup(browser.page_source, 'html.parser')
    data_soup_list.append(data_soup)

    try:
        browser.find_element_by_css_selector(".pageNext.last")
        last_page = True
        print("last page")
    except:
        print("not the last page")
        browser.find_element_by_css_selector(".pageNext").click()
        sleep(5)


browser.quit()
print(f"===fetch:{url} done===")

===fetch:https://rent.591.com.tw/?kind=0&region=1&rentprice=5&shape=2&other=lift,cartplace,cook===
region=1
Page loaded
Page loaded
not the last page
not the last page
not the last page
not the last page
not the last page
not the last page


ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:;" data-first="180" data-total="272" class="pageNext">...</a> is not clickable at point (833, 739). Other element would receive the click: <li class="pull-left infoContent

        

        ">...</li>
  (Session info: chrome=89.0.4389.114)


In [97]:
class leju_crawler:
    def __init__(self):
#         software_names = [SoftwareName.CHROME.value]
#         operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
#         user_agent_rotator = UserAgent(
#             software_names = software_names,
#             operating_systems = operating_systems,
#             limit = 100
#         )

#         self.user_agent = user_agent_rotator.get_random_user_agent()
#         print(user_agent)

        self.user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'

        chrome_options = Options()
        chrome_options.add_argument('--incognito')
        chrome_options.add_argument('headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--window-size=1420,1080')
        chrome_options.add_argument("--disable-gpu")
        # chrome_options.add_argument(f'user-agent={user_agent}')
        # chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])
        # chrome_options.add_argument("--disable-blink-features");
        # chrome_options.add_argument("--disable-blink-features=AutomationControlled");
        # driver = webdriver.Chrome(options=chrome_options)
        executable_path=os.getcwd()+'/chromedriver_6'
#         print(executable_path)

        self.browser = webdriver.Chrome(executable_path=executable_path, options=chrome_options)
    
    
        
    
    def fetch_data(self, url):
        browser = self.browser
        browser.get(url)
        sleep(10)
        data_soup = BeautifulSoup(browser.page_source, 'html.parser')
#         print(data_soup)
        browser.quit()
        
        return data_soup
        
    def get_title(self, data):
        title = data.find('title').string
        
        return title
        
    def get_price_info(self, data):
        avg_price = data.find('div', class_='avg_house_price_val').text
        max_price = data.find('div', class_='max_house_price_val').text
        min_price = data.find('div', class_='min_house_price_val').text
        this_year_avg_price = data.find('div', class_='avg_date_house_price_val').text
        
        info = {
            'avg_price': avg_price,
            'max_price': max_price, 
            'min_price': min_price,
            'this_year_avg_price': this_year_avg_price
        }
        
        return info
    
    def get_basic_info(self, data):
        building_title = data.find('article', class_='building_title').text
        households = data.find('article', class_='households').text
        house_year = data.find('article', class_='house_year').text
        ttotal_floor = data.find('article', class_='ttotal_floor').text
        elementary = data.find('article', class_='elementary').text
        junior = data.find('article', class_='junior').text
        developer1 = data.find('article', class_='developer1').text
        
        basic_info = {
            'building_title': building_title,
            'households': households,
            'house_year': house_year,
            'ttotal_floor': ttotal_floor,
            'elementary': elementary,
            'junior': junior,
            'developer1': developer1,
        }
        
        return basic_info
        
        
    def get_sale_items(self, data):
        items = data.find_all('div', class_='sales-item-box')
        sale_items = []
        for item in items:
            floor = item.find('span').text
            title = item.find('a').text
            link = item.find('a')['href']
            price = item.find_all('li')[1].text
            area = item.find_all('li')[0].text
            
            data = {
                'floor': floor,
                'title': title,
                'link': link,
                'price': price,
                'area': area,
            }
            
            sale_items.append(data)
            
        return sale_items
            
        
            
        
    def get_data_json(self, data):
        title = self.get_title(data)
        price_info = self.get_price_info(data)
        basic_info =self.get_basic_info(data)
        sale_items = self.get_sale_items(data)
        
        data_json = {
            "title": title,
            'price_info': price_info,
            'basic_info': basic_info,
            'sale_items': sale_items
        }
        
        return data_json
        
        
        

In [106]:
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# import os


# class sheet_worker:
#     def get_hash_str(self,data):
#         hash_str = abs(hash(data)) % (10 ** 8)
#         return hash_str
        
#     def get_sheet(self, sheet_key):
#         scopes = ["https://spreadsheets.google.com/feeds"]
#         current_path = os.getcwd()
#         credentials = ServiceAccountCredentials.from_json_keyfile_name(current_path+"/cred.json", scopes)
#         gss_client = gspread.authorize(credentials)
#         spreadsheet_key_path = sheet_key
#         sheet = gss_client.open_by_key(spreadsheet_key_path)
#         return sheet
    
#     def data_to_sheet_value_list(self, data):
#         profile = json.loads(data)
#         title = profile['title']
#         price_info = profile['price_info']
#         basic_info = profile['basic_info']

#         sale_items = profile['sale_items']
#         sheet_value_list = []
#         for sale_item in sale_items:
#             floor = sale_item['floor']
#             sub_title = sale_item['title']
#             link = sale_item['link']
#             price = sale_item['price']
#             area = sale_item['area']

#             sheet_value = [
#                 str(title),
#                 str(price_info),
#                 str(basic_info),
#                 floor,
#                 sub_title,
#                 link,
#                 price,
#                 area
#             ]

#             sheet_value_str = "".join(sheet_value)
#             hash_str = self.get_hash_str(sheet_value_str)
#             sheet_value.append(hash_str)
#             sheet_value_list.append(sheet_value)
            
#         return sheet_value_list
        
        
#     def run(self, data):
#         sheet_key = '15V1XD3p_mD8SSP_TQkY2PwYTM_FjOAXQXD1GuJcrpfI'
#         sheet = self.get_sheet(sheet_key)
#         sheet_bot = sheet.worksheet('bot')

#         profile_list = data['profile']
#         for profile in profile_list:
#             sheet_value_list = self.data_to_sheet_value_list(profile)
#             print(sheet_value_list)
        
#         sheet_row_cnt = 2
#         for sheet_value in sheet_value_list:
#             sheet_bot.insert_row(sheet_value, sheet_row_cnt)
#             sheet_row_cnt +=1
            
            
            
from util.gsheet import gsheet_worker 

In [99]:
url_list = [
    "https://www.leju.com.tw/page_search_result?oid=L37611690f7027",
]
body = {'profile':[]}
for url in url_list:
    crawler = leju_crawler()
    data = crawler.fetch_data(url)
    data_json = crawler.get_data_json(data)
#         data_json = json.loads(data_json)
#         data_json = json.dumps(data_json, indent=4, sort_keys=True, ensure_ascii=False).encode('utf8')
    data_json = json.dumps(data_json, ensure_ascii=False).encode('utf8')
#         print(data_json.decode())
    body['profile'].append(data_json.decode())
    
print(body)

{'profile': ['{"title": "麗軒珍寶-7筆交易紀錄，實價登錄平均成交價為52.33萬/坪  ", "price_info": {"avg_price": "52.33萬/坪", "max_price": "56.16萬/坪", "min_price": "48.96萬/坪", "this_year_avg_price": "--萬/坪"}, "basic_info": {"building_title": "大同街50巷26號", "households": "32戶", "house_year": "19年", "ttotal_floor": "8", "elementary": "文聖國小(雙語)", "junior": "光復國中、江翠國中", "developer1": "珍寶事業"}, "sale_items": [{"floor": "6樓", "title": "自售～麗軒珍寶景觀3...", "link": "https://sale.591.com.tw/home/house/detail/2/8756933.html", "price": "總價2680萬", "area": "坪數47坪"}, {"floor": "2樓", "title": "江翠捷運珍寶三房", "link": "https://buy.yungching.com.tw/house/4587875", "price": "總價2298萬", "area": "坪數35.61坪"}]}']}


In [107]:

sht_worker = gsheet_worker()
sht_worker.run(body)


[['麗軒珍寶-7筆交易紀錄，實價登錄平均成交價為52.33萬/坪  ', "{'avg_price': '52.33萬/坪', 'max_price': '56.16萬/坪', 'min_price': '48.96萬/坪', 'this_year_avg_price': '--萬/坪'}", "{'building_title': '大同街50巷26號', 'households': '32戶', 'house_year': '19年', 'ttotal_floor': '8', 'elementary': '文聖國小(雙語)', 'junior': '光復國中、江翠國中', 'developer1': '珍寶事業'}", '6樓', '自售～麗軒珍寶景觀3...', 'https://sale.591.com.tw/home/house/detail/2/8756933.html', '總價2680萬', '坪數47坪', 35932627], ['麗軒珍寶-7筆交易紀錄，實價登錄平均成交價為52.33萬/坪  ', "{'avg_price': '52.33萬/坪', 'max_price': '56.16萬/坪', 'min_price': '48.96萬/坪', 'this_year_avg_price': '--萬/坪'}", "{'building_title': '大同街50巷26號', 'households': '32戶', 'house_year': '19年', 'ttotal_floor': '8', 'elementary': '文聖國小(雙語)', 'junior': '光復國中、江翠國中', 'developer1': '珍寶事業'}", '2樓', '江翠捷運珍寶三房', 'https://buy.yungching.com.tw/house/4587875', '總價2298萬', '坪數35.61坪', 55442021]]


In [96]:
# item = data.find_all('div', class_='sales-item-box')[1]
# # print(item)
# item.find_all('li')[0].text